<a href="https://colab.research.google.com/github/yvkrishna/vrushak/blob/main/Rain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.32.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

Installing dependencies for Colab environment
TensorFlow version: 2.4.0


In [ ]:
'''
    cloning the repo COVID-19 from github
'''
!git clone https://github.com/Naveena01/Rainfall_prediction.git

Cloning into 'Rainfall_prediction'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [ ]:
 weather=pd.read_csv('/content/Rainfall_prediction/weatherHistory_updated.csv')


In [ ]:
weather.head()

,Summary,Temperature (C),Humidity,Pressure (millibars)
0,Partly Cloudy,9.472222,0.89,1015.13
1,Partly Cloudy,9.355556,0.86,1015.63
2,Mostly Cloudy,9.377778,0.89,1015.94
3,Partly Cloudy,8.288889,0.83,1016.41
4,Mostly Cloudy,8.755556,0.83,1016.51


In [ ]:
weather= weather.drop(weather.index[20000:len(weather)])
weather.shape

(20000, 4)

In [ ]:
target= weather['Summary']
data = weather.drop(['Summary'], axis=1)

In [ ]:
print(data.shape)
print(target.shape)

(20000, 3)
(20000,)


In [ ]:
 class_names = weather['Summary'].unique()
 print(class_names)

['Partly Cloudy' 'Mostly Cloudy' 'Overcast' 'Foggy'
 'Breezy and Mostly Cloudy' 'Clear' 'Breezy and Partly Cloudy'
 'Breezy and Overcast' 'Humid and Mostly Cloudy' 'Humid and Partly Cloudy'
 'Windy and Foggy' 'Windy and Overcast' 'Breezy and Foggy'
 'Windy and Partly Cloudy' 'Breezy' 'Dry and Partly Cloudy'
 'Windy and Mostly Cloudy' 'Dangerously Windy and Partly Cloudy' 'Dry']


In [ ]:
train_limit = int(len(weather)*85/100)
validating_limit = int(len(weather)*10/100)
testing_limit= int(len(weather)*5/100) 

In [ ]:
 training_data = weather.iloc[1:train_limit]
validating_data = weather.iloc[train_limit: len(weather)-testing_limit]
testing_data = weather.iloc[train_limit+validating_limit: len(weather)+1] 

train_target= training_data['Summary']
training_data = training_data.drop(['Summary'], axis=1)

valid_target= validating_data['Summary']
validating_data = validating_data.drop(['Summary'], axis=1)

test_target= testing_data['Summary']
testing_data = testing_data.drop(['Summary'], axis=1)


In [ ]:
print(len(training_data), len(validating_data), len(testing_data))
print(len(training_data)+len(validating_data)+len(testing_data)) 
print(len(training_data)+len(validating_data))
print(len(train_target))
print(len(valid_target))

training_data=pd.concat([training_data,validating_data])
train_target= pd.concat([train_target,valid_target])

print(len(training_data))
print(len(train_target))



16999 2000 1000
19999
18999
16999
2000
18999
18999


In [ ]:
weather.isnull().sum()

Summary                 0
Temperature (C)         0
Humidity                0
Pressure (millibars)    0
dtype: int64

In [ ]:
training_data.dtypes

Temperature (C)         float64
Humidity                float64
Pressure (millibars)    float64
dtype: object

In [ ]:
training_data.head(0)

,Temperature (C),Humidity,Pressure (millibars)


In [ ]:
header = training_data.iloc[0]
training_data=training_data.rename(columns=header)

In [ ]:
training_data.head(0)

,9.355556,0.860000,1015.630000


In [ ]:
train_target.head(1)

1    Partly Cloudy
Name: Summary, dtype: object

In [ ]:
train_target_with_dummies = pd.get_dummies(train_target)
print(train_target_with_dummies)

       Breezy  Breezy and Foggy  ...  Windy and Overcast  Windy and Partly Cloudy
1           0                 0  ...                   0                        0
2           0                 0  ...                   0                        0
3           0                 0  ...                   0                        0
4           0                 0  ...                   0                        0
5           0                 0  ...                   0                        0
...       ...               ...  ...                 ...                      ...
18995       0                 0  ...                   0                        0
18996       0                 0  ...                   0                        0
18997       0                 0  ...                   0                        0
18998       0                 0  ...                   0                        0
18999       0                 0  ...                   0                        0

[18999 rows x 1

In [ ]:
train_target_with_dummies.head()

,Breezy,Breezy and Foggy,Breezy and Mostly Cloudy,Breezy and Overcast,Breezy and Partly Cloudy,Clear,Dangerously Windy and Partly Cloudy,Dry,Dry and Partly Cloudy,Foggy,Humid and Mostly Cloudy,Humid and Partly Cloudy,Mostly Cloudy,Overcast,Partly Cloudy,Windy and Foggy,Windy and Mostly Cloudy,Windy and Overcast,Windy and Partly Cloudy
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
header = train_target_with_dummies.iloc[0]
train_target_with_dummies=train_target_with_dummies.rename(columns=header)

In [ ]:
train_target_with_dummies.head(0)

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((training_data.values, train_target_with_dummies.values))

In [ ]:
for feat, targ in train_dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [9.35555556e+00 8.60000000e-01 1.01563000e+03], Target: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Features: [9.37777778e+00 8.90000000e-01 1.01594000e+03], Target: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Features: [8.28888889e+00 8.30000000e-01 1.01641000e+03], Target: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Features: [8.75555556e+00 8.30000000e-01 1.01651000e+03], Target: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
Features: [9.22222222e+00 8.50000000e-01 1.01666000e+03], Target: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


In [ ]:
train_dataset = train_dataset.shuffle(len(training_data)).batch(1)

In [ ]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(19, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
model = get_compiled_model()
model.fit(train_dataset, epochs=10)

Epoch 1/10
18999/18999 [==============================] - 57s 3ms/step - loss: 0.9978 - accuracy: 0.2929
Epoch 2/10
18999/18999 [==============================] - 55s 3ms/step - loss: 0.1254 - accuracy: 0.3611
Epoch 3/10
18999/18999 [==============================] - 55s 3ms/step - loss: 0.1240 - accuracy: 0.3653
Epoch 4/10
18999/18999 [==============================] - 54s 3ms/step - loss: 0.1222 - accuracy: 0.3740
Epoch 5/10
18999/18999 [==============================] - 54s 3ms/step - loss: 0.1216 - accuracy: 0.3665
Epoch 6/10
18999/18999 [==============================] - 54s 3ms/step - loss: 0.1218 - accuracy: 0.3754
Epoch 7/10
18999/18999 [==============================] - 55s 3ms/step - loss: 0.1215 - accuracy: 0.3712
Epoch 8/10
18999/18999 [==============================] - 54s 3ms/step - loss: 0.1215 - accuracy: 0.3782
Epoch 9/10
18999/18999 [==============================] - 55s 3ms/step - loss: 0.1216 - accuracy: 0.3735
Epoch 10/10
18999/18999 [==============================

In [ ]:
model.input

<KerasTensor: shape=(None, 3) dtype=float64 (created by layer 'dense_input')>

In [ ]:
!mkdir weather_model


mkdir: cannot create directory ‘weather_model’: File exists


Model Saving


In [ ]:
import tempfile

MODEL_DIR = "/content/weather_model"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /content/weather_model/1

INFO:tensorflow:Assets written to: /content/weather_model/1/assets

Saved model:
total 100
drwxr-xr-x 2 root root  4096 Dec 19 14:07 assets
-rw-r--r-- 1 root root 91935 Dec 19 14:08 saved_model.pb
drwxr-xr-x 2 root root  4096 Dec 19 14:08 variables


In [ ]:
!saved_model_cli show --dir /content/weather_model/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1, 3)
        name: serving_default_dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 19)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W1219 14:08:12.699723 140059649578880 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/r

In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update


deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  35457      0 --:--:-- --:--:-- --:--:-- 35457
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRele

Install Tensorflow Serving

In [ ]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 59 not upgraded.
Need to get 223 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.4.0 [223 MB]
Fetched 223 MB in 3s (68.4 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.4.0_all.deb ...
Unpacking tensorflow-model-server (2.4.0) ...
Setting up tensorflow-model-server (2.4.0) ...


In [ ]:
MODEL_DIR

'/content/weather_model'

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=weather_model \
  --model_base_path="$MODEL_DIR" >server.log 2>&1

Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

In [ ]:
test_val = np.array([-1, 9.355556, 0.860000, 1015.630000])
test_val = np.resize(test_val,(1,3))

In [ ]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_val.tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... ing_default", "instances": [[-1.0, 9.355556, 0.86]]}


In [ ]:
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/weather_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

In [ ]:
print(np.max(predictions[0]))
print(np.argmax(predictions[0]))

0.997168481
5


In [ ]:
ground_truth = list(train_target_with_dummies.head(0))
print(ground_truth)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [ ]:
predictions = json.loads(json_response.text)

In [ ]:
print(predictions[0])

[9.73920178e-05, 7.51363768e-05, 0.000241446513, 0.000844836584, 0.000830702134, 0.256503284, 5.42976473e-08, 1.28196233e-22, 1.08774575e-16, 0.0534175783, 3.15342045e-13, 2.23812701e-14, 0.0177928898, 0.600110471, 0.069884561, 4.09376355e-07, 1.07743272e-05, 7.13685222e-05, 0.000119068813]
